In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [4]:
# Determine the number of unique values in each column.
print(application_df.nunique())

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [5]:
# Look at APPLICATION_TYPE value counts for binning
print(application_df['APPLICATION_TYPE'].value_counts())

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < 500].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
print(application_df['CLASSIFICATION'].value_counts())

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()
print(classification_counts[classification_counts > 1])

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = classification_counts[classification_counts < 1883].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [11]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Modify model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Callbacks for early stopping and model checkpoints
early_stopping = EarlyStopping(monitor='val_loss', patience=20, min_delta=0.001, restore_best_weights=True, verbose=1)
model_checkpoint = ModelCheckpoint('AlphabetSoupCharity_Optimization.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

# Train the model
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=200, batch_size=32,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

# Evaluate the optimized model
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Optimized Model Loss: {loss}, Accuracy: {accuracy}")

Epoch 1/200
795/804 [============================>.] - ETA: 0s - loss: 0.5693 - accuracy: 0.7230
Epoch 1: val_accuracy improved from -inf to 0.72198, saving model to AlphabetSoupCharity_Optimization.h5
804/804 [==============================] - 5s 4ms/step - loss: 0.5688 - accuracy: 0.7231 - val_loss: 0.5591 - val_accuracy: 0.7220
Epoch 2/200
790/804 [============================>.] - ETA: 0s - loss: 0.5549 - accuracy: 0.7312
Epoch 2: val_accuracy did not improve from 0.72198
804/804 [==============================] - 2s 3ms/step - loss: 0.5549 - accuracy: 0.7310 - val_loss: 0.5564 - val_accuracy: 0.7217
Epoch 3/200
785/804 [============================>.] - ETA: 0s - loss: 0.5508 - accuracy: 0.7329
Epoch 3: val_accuracy improved from 0.72198 to 0.72362, saving model to AlphabetSoupCharity_Optimization.h5
804/804 [==============================] - 2s 3ms/step - loss: 0.5520 - accuracy: 0.7318 - val_loss: 0.5591 - val_accuracy: 0.7236
Epoch 4/200
778/804 [============================>.]

In [22]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from google.colab import files

# Increasing model complexity and adding dropout and batch normalization
model = tf.keras.models.Sequential([
    Dense(256, activation='relu', input_dim=X_train_scaled.shape[1]),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

optimizer = Adam(learning_rate=0.0005)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)
model_checkpoint = ModelCheckpoint('AlphabetSoupCharity_Optimization.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.0001, verbose=1)

history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=250,
    batch_size=64,
    callbacks=[early_stopping, model_checkpoint, reduce_lr],
    verbose=1
)

loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Further Optimized Model Loss: {loss}, Accuracy: {accuracy}")

files.download('AlphabetSoupCharity_Optimization.h5')

Epoch 1/250
400/402 [============================>.] - ETA: 0s - loss: 0.8286 - accuracy: 0.7031
Epoch 1: val_accuracy improved from -inf to 0.71860, saving model to AlphabetSoupCharity_Optimization.h5
402/402 [==============================] - 6s 9ms/step - loss: 0.8281 - accuracy: 0.7034 - val_loss: 0.7808 - val_accuracy: 0.7186 - lr: 5.0000e-04
Epoch 2/250
398/402 [============================>.] - ETA: 0s - loss: 0.7618 - accuracy: 0.7238
Epoch 2: val_accuracy improved from 0.71860 to 0.71907, saving model to AlphabetSoupCharity_Optimization.h5
402/402 [==============================] - 2s 5ms/step - loss: 0.7619 - accuracy: 0.7234 - val_loss: 0.7317 - val_accuracy: 0.7191 - lr: 5.0000e-04
Epoch 3/250
400/402 [============================>.] - ETA: 0s - loss: 0.7174 - accuracy: 0.7270
Epoch 3: val_accuracy improved from 0.71907 to 0.72198, saving model to AlphabetSoupCharity_Optimization.h5
402/402 [==============================] - 2s 5ms/step - loss: 0.7170 - accuracy: 0.7270 - v

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
from tensorflow.keras.optimizers import SGD

# Define the model architecture
model = tf.keras.models.Sequential([
    Dense(256, activation='relu', input_dim=X_train_scaled.shape[1]),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Use SGD optimizer with momentum
optimizer = SGD(learning_rate=0.01, momentum=0.9)

# Compile the model
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1)
model_checkpoint = ModelCheckpoint('AlphabetSoupCharity_Optimization.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

# Train the model
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=250,
    batch_size=64,
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Model Loss with SGD: {loss}, Accuracy: {accuracy}")

Epoch 1/250
396/402 [============================>.] - ETA: 0s - loss: 0.8296 - accuracy: 0.7091
Epoch 1: val_accuracy improved from -inf to 0.72035, saving model to AlphabetSoupCharity_Optimization.h5
402/402 [==============================] - 5s 9ms/step - loss: 0.8297 - accuracy: 0.7087 - val_loss: 0.7833 - val_accuracy: 0.7203
Epoch 2/250
391/402 [============================>.] - ETA: 0s - loss: 0.7710 - accuracy: 0.7241
Epoch 2: val_accuracy improved from 0.72035 to 0.72070, saving model to AlphabetSoupCharity_Optimization.h5
402/402 [==============================] - 2s 5ms/step - loss: 0.7708 - accuracy: 0.7240 - val_loss: 0.7496 - val_accuracy: 0.7207
Epoch 3/250
402/402 [==============================] - ETA: 0s - loss: 0.7380 - accuracy: 0.7260
Epoch 3: val_accuracy improved from 0.72070 to 0.72338, saving model to AlphabetSoupCharity_Optimization.h5
402/402 [==============================] - 2s 5ms/step - loss: 0.7380 - accuracy: 0.7260 - val_loss: 0.7237 - val_accuracy: 0.

In [21]:
#Modify the Model Architecture

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,LeakyReLU
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train_scaled.shape[1],), kernel_regularizer=l1_l2(l1=0.001, l2=0.001)),
    Dropout(0.2),
    Dense(256, activation='relu', kernel_regularizer=l1_l2(l1=0.001, l2=0.001)),
    Dropout(0.2),
    Dense(128, activation='LeakyReLU', kernel_regularizer=l1_l2(l1=0.001, l2=0.010)),
    Dense(1, activation='sigmoid')
])

optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('AlphabetSoupCharity_Optimized_Model.h5', save_best_only=True)

history = model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=100, callbacks=[early_stopping, model_checkpoint], batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Model Loss with SGD: {loss}, Accuracy: {accuracy}")

Epoch 1/100
644/644 [==============================] - 6s 8ms/step - loss: 7.3843 - accuracy: 0.7142 - val_loss: 3.9860 - val_accuracy: 0.7186
Epoch 2/100
644/644 [==============================] - 6s 9ms/step - loss: 2.3441 - accuracy: 0.7262 - val_loss: 1.3635 - val_accuracy: 0.7190
Epoch 3/100
644/644 [==============================] - 5s 8ms/step - loss: 1.0283 - accuracy: 0.7271 - val_loss: 0.8418 - val_accuracy: 0.7195
Epoch 4/100
644/644 [==============================] - 5s 8ms/step - loss: 0.7644 - accuracy: 0.7264 - val_loss: 0.7207 - val_accuracy: 0.7178
Epoch 5/100
644/644 [==============================] - 6s 9ms/step - loss: 0.6926 - accuracy: 0.7266 - val_loss: 0.6789 - val_accuracy: 0.7168
Epoch 6/100
644/644 [==============================] - 5s 8ms/step - loss: 0.6642 - accuracy: 0.7265 - val_loss: 0.6592 - val_accuracy: 0.7178
Epoch 7/100
644/644 [==============================] - 6s 9ms/step - loss: 0.6481 - accuracy: 0.7278 - val_loss: 0.6487 - val_accuracy: 0.7182